In [ ]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import sys
sys.path.insert(0, "..")

In [ ]:
import utils_analytics as ut
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os 
from pandas.core.common import flatten


# Introduction

Bla bla bla

### Dataset

In [ ]:
tweets_df, account_df = ut.loadData()
tweets_df.head()

### Preprocessing text

Esempio testo tweet originale:

In [ ]:
print(tweets_df.loc[9,'tweet'])

Come è possibile vedere, i tweet presentano solitamente hashtag, menzioni, url ecc.. che però difficilmente si riescono ad encodare come informazioni utili da passare in input ad una LSTM. Pertanto uno step di preprocessing è sicuramente necessario per ridurre il numero di OOV words mantenendo intatte il più possibile le informazioni che questi dati possiedono.

Step di preprocessing per ogni singolo tweet:
- 'RT' -> ' retweet '
- '\n' -> ' '
- '$apple' -> ' stock '
- '@' -> ' email '
- '1,2,3..' -> ' number '
- '$,£..' -> ' money '
- '#' -> ' hashtag '
- '@pontifex' -> ' username '

- 'http,https..' -> 'url'
- 'ahah, haha, ajaj, jaja' -> 'ahah'
- '-' -> ' '
- "'" -> " '"
- Remove tweets too shorts (minimum 3 tokens required)

Perchè lo facciamo così:
- cashtag, money, emoji:
- esclusione tweet corti: abbiamo deciso di eliminare i tweet che dopo il preprocessing possedevano un numero di token inferiore a 3. Questo ha permesso di 'pulire' il dataset da tweet poco esplicativi (anche per un umano) che avrebbero costituito degli outlier e che avrebbero peggiorato le performance
- inglese vs altre lingue: 
- FastText vs Glove:

Testo pulito:

Stampare esempi testo pulito

In [ ]:
dataset_df = pd.read_pickle(ut.DATA_FOLDER / 'processed_dataset_v1.pkl')
dataset_df.head()

In [ ]:
print(dataset_df.loc[9,'processed_tweet'])

### Text of single tweet
- architettura modello
- da testo a embedding 
- tuning hyperparametri 
- motivare : dropout, weight decay, class imbalance  
- risultati

### Text of single tweet + text features 
- Perchè utilizziamo delle feature ( la LSTM non considera troppo elementi statistici del testo come RT, hashtag, numeri di cose, ecc..)
- Come le passiamo: struttura modello + zscore 
- Feature utilizzate:
    - Is retweet? Yes/No
    - N° of URLs, tags, hashtags, cashtag, currency simbols, emails, numbers, emoticons, emojis, stopwords, punctuation
- Perchè queste feature? rilevanza, analisi di correlazione
- Esempi di tweet che mostrano la rilevanza delle feature (i bot tendono a avere più citazioni, hashtag, boh ) risultati 


In [ ]:
singletweet_features_df = pd.read_pickle(ut.DATA_FOLDER / 'processed_dataset_v2.pkl')
singletweet_features_df.head()

In [ ]:
plt.figure(figsize=(20,10))
cor = singletweet_features_df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature selection
def select_features(X_train, y_train, X_test):
	# configure to select all features
	fs = SelectKBest(score_func=f_classif, k='all')
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

In [ ]:
feature_columns = ['is_rt','url_c','tag_c','hashtag_c','cashtag_c','money_c','email_c','number_c','emoji_c','emoticon_c','len_tweet','stopwords_c','punct_c']
train_ds = singletweet_features_df[singletweet_features_df['split'] == 'train'].reset_index(drop=True)
val_ds = singletweet_features_df[singletweet_features_df['split'] == 'val'].reset_index(drop=True)
test_ds = singletweet_features_df[singletweet_features_df['split'] == 'test'].reset_index(drop=True)

X_train = train_ds[feature_columns]
y_train = train_ds['label']

X_test = val_ds[feature_columns]
y_test = val_ds['label']

# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print(f'{i} -> {feature_columns[i]}: {fs.scores_[i]/sum(fs.scores_)*100:.3f}%')
# plot the scores

plt.figure(figsize=(30,10))
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

Come è possibile vedere, la feature 'is_rt' (Is retweet? Yes/No) sia molto correlata al valore della label (Bot/Human). Andiamo pertanto ad analizzare le percentuali che caratterizzano la differenza tra tweet prodotti da bot che sono retweet o post 'nuovi'.

In [ ]:
num_rt_bot = singletweet_features_df[(singletweet_features_df['is_rt'] == 1.0) & (singletweet_features_df['label'] == 1.0)].shape[0]
num_nort_bot = singletweet_features_df[(singletweet_features_df['is_rt'] == 0.0) & (singletweet_features_df['label'] == 1.0)].shape[0]
num_tweets = singletweet_features_df.shape[0]
print(f'Number of tweets from bots which are retweet: {num_rt_bot} - ({num_rt_bot/num_tweets*100:.1f}%)')
print(f'Number of tweets from bots which are not retweet: {num_nort_bot} - ({num_nort_bot/num_tweets*100:.1f}%)\n')

num_rt_human = singletweet_features_df[(singletweet_features_df['is_rt'] == 1.0) & (singletweet_features_df['label'] == 0.0)].shape[0]
num_nort_human = singletweet_features_df[(singletweet_features_df['is_rt'] == 0.0) & (singletweet_features_df['label'] == 0.0)].shape[0]
print(f'Number of tweets from humans which are retweet: {num_rt_human} - ({num_rt_human/num_tweets*100:.1f}%)')
print(f'Number of tweets from humans which are not retweet: {num_nort_human} - ({num_nort_human/num_tweets*100:.1f}%)\n')

print(f"Pearson Correlation:\n{singletweet_features_df[['is_rt','label']].corr()}")

Quindi c'è una probabilità doppia che se il tweet è un retweet l'utente sia in realtà un bot.

Andiamo ora invece ad effettuare lo stesso studio sulla feature 'cashtag_c', che mostra il numero di cashtag all'interno di ogni singolo tweet e che appare molto poco correlata alla label finale.

In [ ]:
print(f"Pearson Correlation:\n{singletweet_features_df[['cashtag_c','label']].corr()}")

Infine andiamo a comparare la media di url utilizzati per singolo tweet da bot e umani con la corrispondente media di hashtag.

In [ ]:
mean_url_bot = singletweet_features_df[singletweet_features_df['label'] == 1.0]['url_c'].mean()
mean_url_nobot = singletweet_features_df[singletweet_features_df['label'] == 0.0]['url_c'].mean()
print(f"Average z-score of URLs per single tweet by bot user: {mean_url_bot:.3f}")
print(f"Average z-score of URLs per single tweet by human user: {mean_url_nobot:.3f}")
print(f"Difference: {abs(mean_url_bot - mean_url_nobot):.3f}")

In [ ]:
mean_hashtag_bot = singletweet_features_df[singletweet_features_df['label'] == 1.0]['hashtag_c'].mean()
mean_hashtag_nobot = singletweet_features_df[singletweet_features_df['label'] == 0.0]['hashtag_c'].mean()
print(f"Average z-score of hashtags per single tweet by bot user: {mean_hashtag_bot:.3f}")
print(f"Average z-score of hashtags per single tweet by human user: {mean_hashtag_nobot:.3f}")
print(f"Difference: {abs(mean_hashtag_bot - mean_hashtag_nobot):.3f}")

Il margine è nettamente più ampio (più del doppio)

### Multi tweets + text features 
- Perchè? Ovviamente da più tweet si capisce meglio e ci sono più informazioni, sia dal punto di vista del testo sia per i metadati, la cui analisi diventa più statisticamente rilevante (num di RT uguali, num avg hashtag per tweet, ecc) 
- Quali features e spiegazione: 
    - rilevanza features (perchè una feature ha senso) 
    - correlazione e esempi presi dal dataset 
- Risultati 


In [ ]:
multitweet_features_df = pd.read_pickle(ut.DATA_FOLDER / 'processed_dataset_v3.pkl')
multitweet_features_df.head()

In [ ]:
plt.figure(figsize=(20,10))
cor = multitweet_features_df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
feature_columns = multitweet_features_df.columns.difference(
	['account_id','label','split','tweet','processed_tweet','n_processed_tweet','n_tweet']).tolist()
train_ds = multitweet_features_df[multitweet_features_df['split'] == 'train'].reset_index(drop=True)
val_ds = multitweet_features_df[multitweet_features_df['split'] == 'val'].reset_index(drop=True)
test_ds = multitweet_features_df[multitweet_features_df['split'] == 'test'].reset_index(drop=True)

X_train = train_ds[feature_columns]
y_train = train_ds['label']

X_test = val_ds[feature_columns]
y_test = val_ds['label']

# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print(f'{i} -> {feature_columns[i]}: {fs.scores_[i]/sum(fs.scores_)*100:.3f}%')
# plot the scores

plt.figure(figsize=(30,10))
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

Analisi numero di retweet per account:

In [ ]:
NUM_TW_FEATURES = 30
NUM_TW_TXT = 10

def retweet_count(proc_sentence : list):
    return proc_sentence.count('retweet')

df = dataset_df.copy(deep=True)

# AGGREGATE TWEET FROM SAME ACCOUNT 
aggregation_functions = {'account_id': 'first', 'tweet': lambda x : x.tolist(), 'label': 'first', 'split': 'first','processed_tweet': lambda x : x.tolist()}
df = df.groupby(df['account_id'],as_index=False,sort=False).agg(aggregation_functions) 
df = df[df['tweet'].map(lambda x: len(x)) >= NUM_TW_FEATURES].reset_index(drop=True)
df['n_processed_tweet'] = df['processed_tweet'].map(lambda x: x[:NUM_TW_FEATURES]).apply(lambda x : list(flatten(x)))
df['rt_count'] = df['n_processed_tweet'].apply(retweet_count)

In [ ]:
mean_rt_bot = df[df['label'] == 1.0]['rt_count'].mean()
mean_rt_nobot = df[df['label'] == 0.0]['rt_count'].mean()
mean_rt = df['rt_count'].mean()
print(f"Average number of retweets: {mean_rt:.3f}")
print(f"Average number of retweets by bot users: {mean_rt_bot:.3f} (+{(mean_rt_bot-mean_rt)/mean_rt*100:.3f}%)")
print(f"Average number of retweets by human users: {mean_rt_nobot:.3f} ({(mean_rt_nobot-mean_rt)/mean_rt*100:.3f}%)")
print(f"Pearson Correlation:\n{multitweet_features_df[['rt_count','label']].corr()}")

Analisi parole diverse per account:

In [ ]:
from string import punctuation
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords',ut.DATA_FOLDER)

sw = stopwords.words('english')

def clean_tweet(tweet: list ):
        to_remove = ['retweet','username','hashtag','url','emoticon','emoji','number','stock','money','email']
        return [x for x in tweet if x not in to_remove and x not in punctuation and x not in sw]

def unique_words_ratio(sentence_list : list[list]):
    s = []
    for sentence in sentence_list:
        if sentence[0] != 'retweet':
            s.extend(clean_tweet(sentence))
    
    if s : return len(set(s)) / len(s)
    else : return 1.0

df['unique_words_ratio'] = df['n_processed_tweet'].apply(unique_words_ratio)

In [ ]:
mean_unique_words_bot = df[df['label'] == 1.0]['unique_words_ratio'].mean()
mean_unique_words_nobot = df[df['label'] == 0.0]['unique_words_ratio'].mean()
mean_unique_words = df['unique_words_ratio'].mean()
print(f"Average unique words used by each account in general: {mean_unique_words:.3f}")
print(f"Average unique words used by each bot: {mean_unique_words_bot:.3f} ({(mean_unique_words_bot-mean_unique_words)/mean_unique_words*100:.3f}%)")
print(f"Average unique words used by each human: {mean_unique_words_nobot:.3f} (+{(mean_unique_words_nobot-mean_unique_words)/mean_unique_words*100:.3f}%)")
print(f"Pearson Correlation:\n{multitweet_features_df[['unique_words_ratio','label']].corr()}")

Esempio utente che posta tweet con parole maggiormente uguali:

In [ ]:
df.loc[df['unique_words_ratio'].idxmin(),'tweet'][:3]

Esempio utente che posta tweet con parole tendenzialmente diverse:

In [ ]:
df.loc[df['unique_words_ratio'][:100].idxmax(),'tweet'][:10]